# **Itay Regev | 203539119**

# **Homework Assignment: Image Segmentation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download the data

[Download](http://cocodataset.org/#download "Download COCO") the following files on to your PC.

## Install and Import Required Libraries

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import random
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## For visualizing results
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

## Dataset Exploration with PyCoCO

In [ ]:
dataDir='./COCOdataset2017'
dataType='val'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

# # initialize the COCO api for instance annotations
# coco=COCO(annFile)


path_to_instances_val_json = "/content/drive/MyDrive/HIT/שנה ב סמסטר ב/עיבוד תמונות ביו-רפואיות/תרגיל בית - להגשה/medium_tutorial/project folder/COCOdataset2017/annotations/instances_val.json"

# initialize the COCO api for instance annotations
coco=COCO(path_to_instances_val_json)


In [ ]:
# display COCO categories and supercategories
catIDs = coco.getCatIds()
cats = coco.loadCats(catIDs)

nms_cats=[cat['name'] for cat in cats]
print(nms_cats)
print(len(nms_cats))
# print(len(nms),'COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print(len(nms),'COCO supercategories: \n{}'.format(' '.join(nms)))

### COCO Class Ids

In [ ]:
category_names_lst = []
category_ids_lst =[]
for i in range(len(cats)):
  category_names_lst.append(cats[i]['name'])
  category_ids_lst.append(cats[i]['id'])

print(category_names_lst)
print(len(category_names_lst))
print(len(category_ids_lst))

In [ ]:
cats = coco.loadCats(catIDs)

def getClassName(classID, cats):
    for i in range(len(cats)):
      print(cats[i])
      if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

print('The class name is', getClassName(92, cats))



#### Class Filtering


In [ ]:
# get all images containing given categories, select one at random
filterClasses = category_names_lst
catIds = coco.getCatIds(catNms=filterClasses);
imgIds = coco.getImgIds(catIds=catIds);

print("Number of images containing all required classes:", len(imgIds))

In [ ]:
# Get images ids by category id.
imgIds = coco.getImgIds(catIds=category_ids_lst[0]);
print("images id of images that belogns to category {}:".format(category_ids_lst[0]),imgIds)

In [ ]:
########## ALl POSSIBLE COMBINATIONS ########
classes = ['laptop', 'tv', 'cell phone','banner', 'building-other', 'metal', 'pavement', 'road', 'sky-other', 'tree', 'other']

images = []
if classes!=None:
    # iterate for each individual class in the list
    for className in classes:
        # get all images containing given class
        catIds = coco.getCatIds(catNms=className)
        imgIds = coco.getImgIds(catIds=catIds)
        images += coco.loadImgs(imgIds)
else:
    imgIds = coco.getImgIds()
    images = coco.loadImgs(imgIds)

# Now, filter out the repeated images
unique_images = []
for i in range(len(images)):
    if images[i] not in unique_images:
        unique_images.append(images[i])

dataset_size = len(unique_images)

print("Number of images containing the filter classes:", dataset_size)

## Create out data generator

In [ ]:
def filterDataset(folder, classes=None, mode='train'):
    # initialize COCO api for instance annotations
    annFile = '{}/annotations/instances_{}.json'.format(folder, mode)
    coco = COCO(annFile)

    images = []
    if classes!=None:
        # iterate for each individual class in the list
        for className in classes:
            # get all images containing given categories
            catIds = coco.getCatIds(catNms=className)
            imgIds = coco.getImgIds(catIds=catIds)
            images += coco.loadImgs(imgIds)

    else:
        imgIds = coco.getImgIds()
        images = coco.loadImgs(imgIds)

    # Now, filter out the repeated images
    unique_images = []
    for i in range(len(images)):
        if images[i] not in unique_images:
            unique_images.append(images[i])

    random.shuffle(unique_images)
    dataset_size = len(unique_images)

    return unique_images, dataset_size, coco

In [ ]:
folder = "/content/drive/MyDrive/HIT/שנה ב סמסטר ב/עיבוד תמונות ביו-רפואיות/תרגיל בית - להגשה/medium_tutorial/project folder/COCOdataset2017"
classes = category_names_lst
mode = 'val'

images, dataset_size, coco = filterDataset(folder, classes,  mode)

In [ ]:
# examine the first element in the list of images object - coco_url, file_name
dict_image_0 = dict(images[0])
print(dict_image_0['coco_url'])
print(dict_image_0['file_name'])
print(images[0])

### Generate the Image and mask

The dataGeneratorCoco function takes 8 inputs-   
(a) **images**: the image list returned by the filterDataset function  
(b) **classes**: the filter classes list  
(c) **coco**: the coco object returned by the filterDataset function  
(d) **folder**: the folder containing the dataset  
(e) **input_image_size**: size of input image to your model (width, height)  
(f) **batch_size**: batch size  
(g) **mode**: 'train' or 'val'  
(h) **mask_type**: 'binary' or 'normal'(classes can't be None)


The function returns a data generator object yielding batches of images and their corresponding masks

In [ ]:
def getClassName(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return None

def getImage(imageObj, img_folder, input_image_size):
    # Read and normalize an image
    train_img = io.imread(img_folder + '/' + imageObj['file_name'])/255.0
    # Resize
    train_img = cv2.resize(train_img, input_image_size)
    if (len(train_img.shape)==3 and train_img.shape[2]==3): # If it is a RGB 3 channel image
        return train_img
    else: # To handle a black and white image, increase dimensions to 3
        stacked_img = np.stack((train_img,)*3, axis=-1)
        return stacked_img

def getNormalMask(imageObj, classes, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    cats = coco.loadCats(catIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        className = getClassName(anns[a]['category_id'], cats)
        pixel_value = classes.index(className)+1
        new_mask = cv2.resize(coco.annToMask(anns[a])*pixel_value, input_image_size)
        train_mask = np.maximum(new_mask, train_mask)

    # Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask

def getBinaryMask(imageObj, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        new_mask = cv2.resize(coco.annToMask(anns[a]), input_image_size)

        #Threshold because resizing may cause extraneous values
        new_mask[new_mask >= 0.5] = 1
        new_mask[new_mask < 0.5] = 0

        train_mask = np.maximum(new_mask, train_mask)

    # Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask


def dataGeneratorCoco(images, classes, coco, folder,
                      input_image_size=(224,224), batch_size=4, mode='train', mask_type='binary'):

    img_folder = '{}/images/{}'.format(folder, mode)
    dataset_size = len(images)
    catIds = coco.getCatIds(catNms=classes)

    c = 0
    while(True):
        img = np.zeros((batch_size, input_image_size[0], input_image_size[1], 3)).astype('float')
        mask = np.zeros((batch_size, input_image_size[0], input_image_size[1], 1)).astype('float')

        for i in range(c, c+batch_size): #initially from 0 to batch_size, when c = 0
            imageObj = images[i]

            ### Retrieve Image ###
            train_img = getImage(imageObj, img_folder, input_image_size)

            ### Create Mask ###
            if mask_type=="binary":
                train_mask = getBinaryMask(imageObj, coco, catIds, input_image_size)

            elif mask_type=="normal":
                train_mask = getNormalMask(imageObj, classes, coco, catIds, input_image_size)

            # Add to respective batch sized arrays
            img[i-c] = train_img
            mask[i-c] = train_mask

        c+=batch_size
        if(c + batch_size >= dataset_size):
            c=0
            random.shuffle(images)
        yield img, mask

In [ ]:
batch_size = 4
input_image_size = (224,224)
mask_type = 'normal'

val_gen = dataGeneratorCoco(images, classes, coco, folder,
                            input_image_size, batch_size, mode, mask_type)

Our generator is finally ready, let's define a function to visualize it.

In [ ]:
def visualizeGenerator(gen):
    img, mask = next(gen)

    fig = plt.figure(figsize=(20, 10))
    outerGrid = gridspec.GridSpec(1, 2, wspace=0.1, hspace=0.1)

    for i in range(2):
        innerGrid = gridspec.GridSpecFromSubplotSpec(2, 2,
                        subplot_spec=outerGrid[i], wspace=0.05, hspace=0.05)

        for j in range(4):
            ax = plt.Subplot(fig, innerGrid[j])
            if(i==1):
                ax.imshow(img[j])
            else:
                ax.imshow(mask[j][:,:,0])

            ax.axis('off')
            fig.add_subplot(ax)
    plt.show()

## Thresholding

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

# Initialize lists for predictions, images, and masks
pred_lst = []
img_lst = []
mask_lst = []

# Initialize lists to store metrics
jaccard_indices = []
precisions = []
recalls = []
f1_scores = []

# 141
for i in range(25):
    img, mask = next(val_gen)
    img_lst.append(img)
    mask_lst.append(mask)

    for j in range(img.shape[0]):  # Iterate through each image in the batch
        cur_image = img[j]

        # Convert the image to uint8 if it's not already
        if cur_image.dtype != np.uint8:
            cur_image = (cur_image * 255).astype(np.uint8)

        # Convert each image to grayscale
        gray_cur_image = cv2.cvtColor(cur_image, cv2.COLOR_BGR2GRAY)

        # Apply a threshold
        threshold_value = 128
        _, binary_image = cv2.threshold(gray_cur_image, threshold_value, 255, cv2.THRESH_BINARY)

        pred_lst.append(binary_image)

print("len image list - batch of 4 images",len(img_lst))
print("len mask list - batch of 4 images",len(mask_lst))
print("len pred list",len(pred_lst))

def compute_metrics(true_mask, pred_mask):
    # Flatten the masks
    true_mask = true_mask.flatten()
    pred_mask = pred_mask.flatten()

    # Compute Dice Coefficient
    intersection = np.sum(true_mask * pred_mask)

    # Compute Jaccard Index (Intersection over Union)
    jaccard_index = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(np.logical_or(true_mask, pred_mask))

    # Compute Precision
    precision = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(pred_mask)

    # Compute Recall
    recall = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(true_mask)

    # Compute F-measure (F1 Score)
    f1 = 2 * (precision * recall) / (precision + recall)

    return jaccard_index, precision, recall, f1

# Extracting the first slice of the image, mask, and prediction
counter_images = 0
for i in range(25):
    for j in range(img_lst[i].shape[0]):  # Iterate through each image in the batch
        image = img_lst[i][j]  # Selecting the image slice
        mask_0 = mask_lst[i][j][:,:,0]  # True mask
        pred_0 = pred_lst[counter_images]  # Predicted mask

        # Compute the metrics
        jaccard, precision, recall, f1 = compute_metrics(mask_0, pred_0)

        # Append metrics to the lists

        if jaccard is None:
          jaccard = 0
        if precision is None:
          precision = 0
        if recall is None:
          recall = 0
        if f1 is None:
          f1 = 0

        jaccard_indices.append(jaccard)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f'Image {counter_images + 1}:')
        print(f'Jaccard Index: {jaccard:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F-measure: {f1:.4f}')
        print('---------------------------')

        # Plotting
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # Displaying the image
        axes[0].imshow(image)
        axes[0].set_title('Image')
        axes[0].axis('off')

        # Displaying the true mask
        axes[1].imshow(mask_0, cmap='gray')
        axes[1].set_title('True Mask')
        axes[1].axis('off')

        # Displaying the predicted mask
        axes[2].imshow(pred_0, cmap='gray')
        axes[2].set_title('Predicted Mask')
        axes[2].axis('off')

        plt.show()

        counter_images += 1


# Compute average metrics
average_jaccard = np.mean(jaccard_indices)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Metric': ['Jaccard Index', 'Precision', 'Recall', 'F-measure'],
    'Average Value': [average_jaccard, average_precision, average_recall, average_f1]
})



In [ ]:

print("Thresholding results", "\n", results)

## Edge-based segmentation

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

# Initialize lists for predictions, images, and masks
pred_lst = []
img_lst = []
mask_lst = []

# Initialize lists to store metrics
jaccard_indices = []
precisions = []
recalls = []
f1_scores = []

# 141
for i in range(25):
    img, mask = next(val_gen)
    img_lst.append(img)
    mask_lst.append(mask)

    for j in range(img.shape[0]):  # Iterate through each image in the batch
        cur_image = img[j]

        # Convert the image to uint8 if it's not already
        if cur_image.dtype != np.uint8:
            cur_image = (cur_image * 255).astype(np.uint8)

        # Convert each image to grayscale
        gray_cur_image = cv2.cvtColor(cur_image, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection for edge-based segmentation
        edges = cv2.Canny(gray_cur_image, 100, 200)

        pred_lst.append(edges)

print("len image list - batch of 4 images", len(img_lst))
print("len mask list - batch of 4 images", len(mask_lst))
print("len pred list", len(pred_lst))

def compute_metrics(true_mask, pred_mask):
    # Flatten the masks
    true_mask = true_mask.flatten()
    pred_mask = pred_mask.flatten()



    # Compute Jaccard Index (Intersection over Union)
    jaccard_index = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(np.logical_or(true_mask, pred_mask))

    # Compute Precision
    precision = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(pred_mask)

    # Compute Recall
    recall = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(true_mask)

    # Compute F-measure (F1 Score)
    f1 = 2 * (precision * recall) / (precision + recall)

    return jaccard_index, precision, recall, f1

# Extracting the first slice of the image, mask, and prediction
counter_images = 0
for i in range(25):
    for j in range(img_lst[i].shape[0]):  # Iterate through each image in the batch
        image = img_lst[i][j]  # Selecting the image slice
        mask_0 = mask_lst[i][j][:,:,0]  # True mask
        pred_0 = pred_lst[counter_images]  # Predicted mask

        # Compute the metrics
        jaccard, precision, recall, f1 = compute_metrics(mask_0, pred_0)


        if jaccard is None:
          jaccard = 0
        if precision is None:
          precision = 0
        if recall is None:
          recall = 0
        if f1 is None:
          f1 = 0

        # Append metrics to the lists
        jaccard_indices.append(jaccard)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f'Image {counter_images + 1}:')

        print(f'Jaccard Index: {jaccard:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F-measure: {f1:.4f}')
        print('---------------------------')

        # Plotting
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # Displaying the image
        axes[0].imshow(image)
        axes[0].set_title('Image')
        axes[0].axis('off')

        # Displaying the true mask
        axes[1].imshow(mask_0, cmap='gray')
        axes[1].set_title('True Mask')
        axes[1].axis('off')

        # Displaying the predicted mask
        axes[2].imshow(pred_0, cmap='gray')
        axes[2].set_title('Predicted Mask')
        axes[2].axis('off')

        plt.show()

        counter_images += 1

# Compute average metrics
average_jaccard = np.mean(jaccard_indices)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Metric': ['Jaccard Index', 'Precision', 'Recall', 'F-measure'],
    'Average Value': [average_jaccard, average_precision, average_recall, average_f1]
})



In [ ]:
print("results - edge-based segmentation", "\n", results)

## Clustering-based image segmentation using K-Means

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

# Initialize lists for predictions, images, and masks
pred_lst = []
img_lst = []
mask_lst = []

# Initialize lists to store metrics
jaccard_indices = []
precisions = []
recalls = []
f1_scores = []

# 141
for i in range(25):
    img, mask = next(val_gen)
    img_lst.append(img)
    mask_lst.append(mask)

    for j in range(img.shape[0]):  # Iterate through each image in the batch
        cur_image = img[j]

        # Convert the image to uint8 if it's not already
        if cur_image.dtype != np.uint8:
            cur_image = (cur_image * 255).astype(np.uint8)

        # Convert each image to grayscale
        gray_cur_image = cv2.cvtColor(cur_image, cv2.COLOR_BGR2GRAY)

        # Reshape the image to a 2D array of pixels
        pixel_values = gray_cur_image.reshape((-1, 1))
        pixel_values = np.float32(pixel_values)

        # Define criteria and apply KMeans
        k = 2  # Number of clusters
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(pixel_values)

        # Convert the labels to an image
        segmented_image = kmeans.labels_.reshape(gray_cur_image.shape)

        # Threshold the segmented image to get binary mask
        pred_mask = np.where(segmented_image == 1, 255, 0).astype(np.uint8)

        pred_lst.append(pred_mask)

print("len image list - batch of 4 images", len(img_lst))
print("len mask list - batch of 4 images", len(mask_lst))
print("len pred list", len(pred_lst))

def compute_metrics(true_mask, pred_mask):
    # Flatten the masks
    true_mask = true_mask.flatten()
    pred_mask = pred_mask.flatten()


    # Compute Jaccard Index (Intersection over Union)
    jaccard_index = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(np.logical_or(true_mask, pred_mask))

    # Compute Precision
    precision = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(pred_mask)

    # Compute Recall
    recall = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(true_mask)

    # Compute F-measure (F1 Score)
    f1 = 2 * (precision * recall) / (precision + recall)

    return jaccard_index, precision, recall, f1

# Extracting the first slice of the image, mask, and prediction
counter_images = 0
for i in range(25):
    for j in range(img_lst[i].shape[0]):  # Iterate through each image in the batch
        image = img_lst[i][j]  # Selecting the image slice
        mask_0 = mask_lst[i][j][:,:,0]  # True mask
        pred_0 = pred_lst[counter_images]  # Predicted mask

        # Compute the metrics
        jaccard, precision, recall, f1 = compute_metrics(mask_0, pred_0)


        if jaccard is None:
          jaccard = 0
        if precision is None:
          precision = 0
        if recall is None:
          recall = 0
        if f1 is None:
          f1 = 0

        # Append metrics to the lists
        jaccard_indices.append(jaccard)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f'Image {counter_images + 1}:')

        print(f'Jaccard Index: {jaccard:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F-measure: {f1:.4f}')
        print('---------------------------')

        # Plotting
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # Displaying the image
        axes[0].imshow(image)
        axes[0].set_title('Image')
        axes[0].axis('off')

        # Displaying the true mask
        axes[1].imshow(mask_0, cmap='gray')
        axes[1].set_title('True Mask')
        axes[1].axis('off')

        # Displaying the predicted mask
        axes[2].imshow(pred_0, cmap='gray')
        axes[2].set_title('Predicted Mask')
        axes[2].axis('off')

        plt.show()

        counter_images += 1

# Compute average metrics

average_jaccard = np.mean(jaccard_indices)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Metric': ['Jaccard Index', 'Precision', 'Recall', 'F-measure'],
    'Average Value': [average_jaccard, average_precision, average_recall, average_f1]
})

# Save the results to a CSV file
results.to_csv('result_kmeans.csv', index=False)

print("Results saved to result_kmeans.csv")


In [ ]:
print("results kmeans", "\n", results)

## Clustering-based image segmentation using mean-shift

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

# Initialize lists for predictions, images, and masks
pred_lst = []
img_lst = []
mask_lst = []

# Initialize lists to store metrics
jaccard_indices = []
precisions = []
recalls = []
f1_scores = []

# 141
for i in range(25):
    img, mask = next(val_gen)
    img_lst.append(img)
    mask_lst.append(mask)

    for j in range(img.shape[0]):  # Iterate through each image in the batch
        cur_image = img[j]

        # Convert the image to uint8 if it's not already
        if cur_image.dtype != np.uint8:
            cur_image = (cur_image * 255).astype(np.uint8)

        # Convert each image to grayscale
        gray_cur_image = cv2.cvtColor(cur_image, cv2.COLOR_BGR2GRAY)

        # Apply Mean Shift algorithm for segmentation
        # Convert the image to the correct format
        shifted = cv2.pyrMeanShiftFiltering(cur_image, sp=21, sr=51)

        # Convert the mean-shifted image to grayscale
        gray_shifted = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)

        # Apply a threshold to get a binary mask
        _, binary_image = cv2.threshold(gray_shifted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        pred_lst.append(binary_image)

print("len image list - batch of 4 images", len(img_lst))
print("len mask list - batch of 4 images", len(mask_lst))
print("len pred list", len(pred_lst))

def compute_metrics(true_mask, pred_mask):
    # Flatten the masks
    true_mask = true_mask.flatten()
    pred_mask = pred_mask.flatten()


    # Compute Jaccard Index (Intersection over Union)
    jaccard_index = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(np.logical_or(true_mask, pred_mask))

    # Compute Precision
    precision = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(pred_mask)

    # Compute Recall
    recall = np.sum(np.logical_and(true_mask, pred_mask)) / np.sum(true_mask)

    # Compute F-measure (F1 Score)
    f1 = 2 * (precision * recall) / (precision + recall)

    return jaccard_index, precision, recall, f1

# Extracting the first slice of the image, mask, and prediction
counter_images = 0
for i in range(25):
    for j in range(img_lst[i].shape[0]):  # Iterate through each image in the batch
        image = img_lst[i][j]  # Selecting the image slice
        mask_0 = mask_lst[i][j][:,:,0]  # True mask
        pred_0 = pred_lst[counter_images]  # Predicted mask

        # Compute the metrics
        jaccard, precision, recall, f1 = compute_metrics(mask_0, pred_0)


        if jaccard is None:
          jaccard = 0
        if precision is None:
          precision = 0
        if recall is None:
          recall = 0
        if f1 is None:
          f1 = 0

        # Append metrics to the lists

        jaccard_indices.append(jaccard)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f'Image {counter_images + 1}:')

        print(f'Jaccard Index: {jaccard:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F-measure: {f1:.4f}')
        print('---------------------------')

        # Plotting
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # Displaying the image
        axes[0].imshow(image)
        axes[0].set_title('Image')
        axes[0].axis('off')

        # Displaying the true mask
        axes[1].imshow(mask_0, cmap='gray')
        axes[1].set_title('True Mask')
        axes[1].axis('off')

        # Displaying the predicted mask
        axes[2].imshow(pred_0, cmap='gray')
        axes[2].set_title('Predicted Mask')
        axes[2].axis('off')

        plt.show()

        counter_images += 1

# Compute average metrics

average_jaccard = np.mean(jaccard_indices)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Metric': ['Jaccard Index', 'Precision', 'Recall', 'F-measure'],
    'Average Value': [average_jaccard, average_precision, average_recall, average_f1]
})

# Save the results to a CSV file
results.to_csv('result_mean-shift.csv', index=False)

print("Results saved to result_mean-shift.csv")


In [ ]:
print("results mean-shift", "\n", results)